## Import

In [1]:
# import trimesh as tm
# import topogenesis as tg
import numpy as np

# import os
import math
import ortools
import cvxopt
import numpy

from ortools.linear_solver import pywraplp
from cvxopt import matrix
from cvxopt.blas import dot
from cvxopt.solvers import qp
from cvxopt import solvers

import pylab
import matplotlib.pyplot as plt

## Loading context

In [2]:
# path = os.path.relpath('../data/my_envelope.obj') # building outer boundaries file path
# buildingplot = tm.load(path) # load specified building boundary mesh (footprint extruded to max height TODO: will be USER INPUT)

vox_size = 10
fh = 3.6 # floor height

# max_extents_height = abs(buildingplot.bounds[0][2]) + abs(buildingplot.bounds[1][2])
max_extents_height = 36
# max_extents_width = abs(buildingplot.bounds[0][0]) + abs(buildingplot.bounds[1][0])
max_extents_width = 140
# max_extents_length = abs(buildingplot.bounds[0][1]) + abs(buildingplot.bounds[1][1])
max_extents_length = 150

## FSI presets

In [3]:
# base = buildingplot.apply_transform(tm.transformations.projection_matrix((0,0,0), (0,0,-1))) # project mesh on z plane to get footprint <-- TODO: easier way?
# plot_area = base.area/2 # area calculates both sides of the flattened mesh, we need only 1
plot_area = 10000
FSI_req = 5.0 # goal for the FSI TODO: USER INPUT

## PV presets

In [4]:
# Annual solar energy output of PV is calculated as follows:
# E(kwh) = A*r*H*PR where A = total solar panel area, r = solar panel yield/efficiency, H = Annual average solar radiation, PR = performance ratio
# from this site TODO find better source: https://photovoltaic-software.com/principle-ressources/how-calculate-solar-energy-power-pv-systems 

r = 0.15 # yield/efficiency estimate
H = 1050 # to be updated, currently wikipedia estimate. kWh.y
PR = 0.75 # default value, estimate performance ratio influenced by angle
PV_req = 200 # kWh/m2/year TODO: change to actual expected value from included colors + find source on this value

In [5]:
# c1 = plot_area # maximum area
# # c1 = 100000000000
# c2 = -r * H * PR # PV_constant # update
# c2 = -120
# c3 = 2000000 # PV_req # PV_goal # update to be total req value for a year for the whole building
# c4 = abs(max_extents_height/fh) # floor upper limit
# # c4 = 1000000000 # floor upper limit
# c5 = -1.0 # floor lower limit
# c6 = -100.0 # area lower limit
# c7 = FSI_req # FSI_goal

# TODO: change the constraints to produce sensible results --> work out 1 case by hand

In [35]:
c1 = plot_area # maximum area
c2 = r * H * PR # PV yield per m2 roof
c3 = 200000.0 # PV goal for a year
c4 = 10.0 # floor upper limit
c5 = 1.0 # floor lower limit
c6 = 100.0 # area lower limit
c7 = 10.0 # FSI goal

In [36]:
P = np.array([[0,-(c1**-1*c7**-1)],[-(c1**-1*c7**-1),0]]) # 
q = np.array([0, -c2*c3**-1]) # 
G = np.array([[-1,0],[0,-1],[1,0],[0,1]])
h = np.array([c5, c6, c4, c1])

sol = solvers.qp(
    matrix(P, tc='d'),
    matrix(q, tc='d'),
    matrix(G, tc='d'),
    matrix(h, tc='d'),
    solver="MOSEK"
)

print(sol['x'], sol["primal objective"])

# Solution always maximizes both floors and area!

     pcost       dcost       gap    pres   dres
 0: -3.1476e+00 -6.5607e+04  7e+04  0e+00  4e-12
 1: -3.1496e+00 -6.7982e+02  7e+02  1e-16  6e-13
 2: -3.3400e+00 -2.9464e+01  3e+01  1e-16  3e-14
 3: -6.3534e+00 -9.6716e+00  3e+00  2e-19  2e-15
 4: -6.5918e+00 -7.0740e+00  5e-01  2e-16  2e-16
 5: -6.9006e+00 -6.9165e+00  2e-02  2e-16  3e-17
 6: -6.9062e+00 -6.9064e+00  2e-04  5e-16  3e-17
 7: -6.9062e+00 -6.9063e+00  2e-06  4e-16  3e-17
Optimal solution found.
[ 1.00e+01]
[ 1.00e+04]
 -6.906249434546544


In [37]:
# P = matrix(np.diag([math.sqrt(c1**-1), math.sqrt(c1**-1)]), tc='d') # 2D matrix correct?
# P = matrix(np.array([[0,((c1**-1)*(c7**-1))/2],[((c1**-1)*(c7**-1))/2,0]]), tc='d') # new version test
P = np.array([[0,-((c7**-1)*(c1**-1))/2],[-((c7**-1)*(c1**-1))/2,0]]) # new version test --> negative objective
# P = matrix(np.array([[0,((c1**-1)*(c7**-1))/2],[((c1**-1)*(c7**-1))/2,0]]), tc='d') # negative objective??
q = np.array([0, -c2*c3**-1]) # new version test --> negative objective
G = np.array([[-1,0],[0,-1],[1,0],[0,1]])
h = np.array([c5, c6, c4, c1])

#np.divide(G, c1)
#np.divide(h, c1)

from cvxopt import solvers
sol = solvers.qp(
    matrix(P, tc='d'),
    matrix(q, tc='d'),
    matrix(G, tc='d'),
    matrix(h, tc='d'),
)

print(sol['x'], sol["primal objective"])
# gives 10 floors, -10 as area for optimal solution
# print(P)

     pcost       dcost       gap    pres   dres
 0: -3.0353e+00 -6.5607e+04  7e+04  0e+00  3e-12
 1: -3.0372e+00 -6.7897e+02  7e+02  3e-16  3e-13
 2: -3.2144e+00 -2.8683e+01  3e+01  2e-16  5e-15
 3: -6.1091e+00 -9.2785e+00  3e+00  2e-16  3e-16
 4: -6.1963e+00 -6.5304e+00  3e-01  2e-16  6e-17
 5: -6.4002e+00 -6.4247e+00  2e-02  2e-16  7e-18
 6: -6.4062e+00 -6.4064e+00  2e-04  2e-16  7e-18
 7: -6.4062e+00 -6.4063e+00  2e-06  2e-16  1e-17
Optimal solution found.
[ 1.00e+01]
[ 1.00e+04]
 -6.406249383298748


## ORTools - variables

In [9]:
solver = pywraplp.Solver.CreateSolver("GLOP") # GLOP and SCIP get different and sometimes 'wrong' results, also different results at different times (restarting kernel seems to help)
# TODO:--> different solvers? GUROBI? CP-SAT? constraint programming not MILP?
infinity = math.inf

a = solver.IntVar(1.0, infinity, 'floors') # number of floors
b = solver.IntVar(10.0, infinity, 'length') # building length
c = solver.IntVar(10.0, infinity, 'width') # building width
# d = solver.IntVar(0.0, 90.0, 'rotation') # building orientation

print(solver.NumVariables())

3


## ORTools - constraints

In [10]:
solver.Add(a * fh <= max_extents_height) # ensures height limit is not exceeded

# not sure if these work as it they are supposed to right now:
# solver.Add(b * math.cos(d.solution_value()) + c * math.sin(d.solution_value()) <= max_extents_length) # constrains the length for the given oreientation
# solver.Add(c * math.cos(d.solution_valuen()) + b * math.n(d.solution_extents_width) # constrains the width for the given orientation

print(solver.NumConstraints())

1


## objective function

In [11]:
def FSI_lowres(floors, width, length, plot = plot_area, goal = FSI_req):
    FSI = (floors * width * length) / plot
    FSI_normalized = 1/(1+(FSI - goal)**2)
    if FSI >= goal:
        FSI_normalized = 1 # if FSI is more than needed, normalized value is 1
    return FSI, FSI_normalized # gives a rough estimate of the FSI, and normalizes this value on 0-1 depending on how close to the goal it is. 1 if it exceeds the goal

print(FSI_lowres(10,10,10), FSI_lowres(12,10,9))

(0.1, 0.03998400639744101) (0.108, 0.040109637286945624)


## ORTools - solving

In [12]:
solver.Maximize(a)
# solver.Maximize(FSI_lowres(a.solution_value(), b.solution_value(), c.solution_value())[1]) # <-- gives a result but uses only the lower bound of the variables and stops 
# solver.Maximize(a*b) # <-- this one gives type error (it probably cant pass 'variable' type to the FSI_lowres definition)
# solver.Maximize(FSI_lowres(a,b,c)[1]) # <-- this is what I want in the end

# I might need to use MPSolverParameters.GetIntegerParam to pass current values to the FSI definition, but I don't understand the documentation on this: 
# paras = pywraplp.MPSolverParameters().GetIntegerParam() 

# another thing that may be the root cause of the issue is that by definition multiplying variables in the objective is not possible since then it is no longer a linear problem?
# this would mean I need to TODO: linearize the objective function for FSI (and PV)?

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('floors =', a.solution_value())
    print('width =', b.solution_value())
    print('length =', c.solution_value())
    # print('rotation =', d.solution_value())
else:
    print('The problem does not have an optimal solution.', status, pywraplp.Solver.OPTIMAL)

Solution:
Objective value = 10.0
floors = 10.0
width = 10.0
length = 10.0
